# Курсовой проект "Рекомедательные системы"

**Основное**
- Дедлайн - 14 января 23:59
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)
- Минимальный скор на Тесте 0.18 (retail_test.csv)

**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)  
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель - следите за валидацией 

## Постановка задачи

Необходимо:

Сделать бейзлайны
Сделать модель, подобрать оптимальные параметры
Для каждого юзера оставить по 5 рекомендаций
Достичь целевой метрики precision@5 ≈ 0.25 на retail_test1
! Исключить холодных пользователей

Загрузка библиотек

In [86]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares as als
from implicit.bpr import BayesianPersonalizedRanking as bpr
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
from implicit.nearest_neighbours import all_pairs_knn, NearestNeighboursScorer

# Модель второго уровня
from catboost import CatBoostClassifier

#import os, sys
#sys.path.insert(1, os.getcwd() + '/src/')

#from src.metrics import precision_at_k, recall_at_k
#from src.utils import prefilter_items, postfilter_items, make_unique_recommendations
#from src.recommenders import MainRecommender

In [87]:
import numpy as np
from numpy import bincount, log, log1p, sqrt
from scipy.sparse import coo_matrix, csr_matrix

#from ._nearest_neighbours import NearestNeighboursScorer, all_pairs_knn
#from .recommender_base import RecommenderBase
#from .utils import _batch_call


In [88]:
class ItemItemRecommender:
    """Base class for Item-Item Nearest Neighbour recommender models
    here.
    Parameters
    ----------
    K : int, optional
        The number of neighbours to include when calculating the item-item
        similarity matrix
    num_threads : int, optional
        The number of threads to use for fitting the model. Specifying 0
        means to default to the number of cores on the machine.
    """

    def __init__(self, K=20, num_threads=0):
        self.similarity = None
        self.K = K
        self.num_threads = num_threads
        self.scorer = None

    def fit(self, weighted, show_progress=True):
        """Computes and stores the similarity matrix"""
        self.similarity = all_pairs_knn(
            weighted, self.K, show_progress=show_progress, num_threads=self.num_threads
        ).tocsr()
        self.scorer = NearestNeighboursScorer(self.similarity)

    def recommend(
        self,
        userid,
        user_items,
        N=10,
        filter_already_liked_items=True,
        filter_items=None,
        recalculate_user=False,
        items=None,
    ):
        """returns the best N recommendations for a user given its id"""
        if not np.isscalar(userid):
            return _batch_call(
                self.recommend,
                userid,
                user_items=user_items,
                N=N,
                filter_already_liked_items=filter_already_liked_items,
                filter_items=filter_items,
                recalculate_user=recalculate_user,
                items=items,
            )

        if (filter_already_liked_items or recalculate_user) and not isinstance(
            user_items, csr_matrix
        ):
            raise ValueError("user_items needs to be a CSR sparse matrix")

        if userid >= user_items.shape[0]:
            raise ValueError("userid is out of bounds of the user_items matrix")

        if filter_items is not None and items is not None:
            raise ValueError("Can't specify both filter_items and items")

        if filter_items is not None:
            N += len(filter_items)
        elif items is not None:
            items = np.array(items)
            N = self.similarity.shape[0]
            # check if items contains itemids that are not in the model(user_items)
            if items.max() >= N or items.min() < 0:
                raise IndexError("Some of selected itemids are not in the model")

        ids, scores = self.scorer.recommend(
            userid,
            user_items.indptr,
            user_items.indices,
            user_items.data,
            K=N,
            remove_own_likes=filter_already_liked_items,
        )

        if filter_items is not None:
            mask = np.in1d(ids, filter_items, invert=True)
            ids, scores = ids[mask][:N], scores[mask][:N]

        elif items is not None:
            mask = np.in1d(ids, items)
            ids, scores = ids[mask], scores[mask]

            # returned items should be equal to input selected items
            missing = items[np.in1d(items, ids, invert=True)]
            if missing.size:
                ids = np.append(ids, missing)
                scores = np.append(scores, np.full(missing.size, -np.finfo(scores.dtype).max))

        return ids, scores

In [89]:
def recall_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    if k < len(recommended_list):
        recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)

    return recall

def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    

    bought_list = bought_list  # Тут нет [:k] !!
    
    if k < len(recommended_list):
        recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision

In [90]:
def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

    top_popular = popularity[popularity['share_unique_users'] > 0.2].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.02].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]

    # Уберем товары, которые не продавались за последние 12 месяцев

    # Уберем не интересные для рекоммендаций категории (department)
    if item_features is not None:
        department_size = pd.DataFrame(item_features.\
                                        groupby('department')['item_id'].nunique().\
                                        sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]


    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]

    # Уберем слишком дорогие товарыs
    data = data[data['price'] < 50]

    # Возбмем топ по популярности
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()
    
    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999
    
    # ...

    return data

In [91]:
class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()

        self.user_item_matrix = self._prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self._prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def _prepare_matrix(data):
        """Готовит user-item матрицу"""
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',  # Можно пробовать другие варианты
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

        return user_item_matrix

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = als(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    def _update_dict(self, user_id):
        """Если появился новыю user / item, то нужно обновить словари"""

        if user_id not in self.userid_to_id.keys():

            max_id = max(list(self.userid_to_id.values()))
            max_id += 1

            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})

    def _get_similar_item(self, item_id):
        """Находит товар, похожий на item_id"""
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)  # Товар похож на себя -> рекомендуем 2 товара
        top_rec = recs[1][0]  # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]

    def _extend_with_top_popular(self, recommendations, N=5):
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""

        if len(recommendations) < N:
            recommendations.extend(self.overall_top_purchases[:N])
            recommendations = recommendations[:N]

        return recommendations
    
    def _get_recommendations(self, user, model, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        res = [self.id_to_itemid[rec[0]] for rec in model.recommend(userid=self.userid_to_id[user],
                                        user_items=csr_matrix(self.user_item_matrix).tocsr(),
                                        N=N,
                                        filter_already_liked_items=False,
                                        filter_items=[self.itemid_to_id[999999]],
                                        recalculate_user=True)]

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_als_recommendations(self, user, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.model, N=N)

    def get_own_recommendations(self, user, N=5):
        """Рекомендуем товары среди тех, которые юзер уже купил"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.own_recommender, N=N)

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        top_users_purchases = self.top_purchases[self.top_purchases['user_id'] == user].head(N)

        res = top_users_purchases['item_id'].apply(lambda x: self._get_similar_item(x)).tolist()
        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        res = []

        # Находим топ-N похожих пользователей
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)
        similar_users = [rec[0] for rec in similar_users]
        similar_users = similar_users[1:]   # удалим юзера из запроса

        for user in similar_users:
            res.extend(self.get_own_recommendations(user, N=1))

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

Загрузка данных

In [92]:
DATASET_PATH = '/Users/ekaterina/Desktop/LEARN/IT/Рекомендательные системы/retail_train.csv'
ITEM_FEATURES_PATH = '/Users/ekaterina/Desktop/LEARN/IT/Рекомендательные системы/2_product.csv'
USER_FEATURES_PATH = '/Users/ekaterina/Desktop/LEARN/IT/Рекомендательные системы/2_hh_demographic.csv'

In [93]:
data = pd.read_csv(DATASET_PATH)
item_features = pd.read_csv(ITEM_FEATURES_PATH)
user_features = pd.read_csv(USER_FEATURES_PATH)

#### Описание датасета:

user_id - id покупателя

backet_id - номер чека

item_id - id товара

quantity - количество конкретного товара в одной покупке

sales_value - стоимость покупkи, долл

store_id - id магазина, где была совершена покупка

retail_disc - скидка магазина

trans_time - время покупки (транзакции)

week_no - номер недели, когда была покупка

coupon_disc - скидка по купону

coupon_match_disc - дополнительная скидка

#### Подготовка данных

In [94]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'

In [95]:
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

In [96]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [97]:
user_features.head(3)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8


Опишем полезные функции

In [98]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")
    
def make_recommendations(df_result, recommend_model, N_PREDICT=50, USER_COL='user_id'):
    return df_result[USER_COL].apply(lambda x: recommend_model(x, N=N_PREDICT))

def calc_recall_at_k(df_data, top_k, ACTUAL_COL='actual'):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()
        
def calc_precision_at_k(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()
        
def rerank(user_id, df, USER_COL='user_id', proba_col_name='proba_item_purchase', N=5):
    return df[df[USER_COL]==user_id].sort_values(proba_col_name, ascending=False).head(N).item_id.tolist()

def get_scores(df_result, recommend_model, N_PREDICT=50, USER_COL='user_id'):
    return df_result[USER_COL].apply(lambda x: recommend_model(x, N=N_PREDICT))

Train test split

Делим датасет на 3 части:
1) обучающий для модели 1 уровня
2) валидационный для модели 1 уровня = обучающий для модели 2 уровня
3) валидационный для модели 2 уровня

Модель 1 уровня - MATCHER (сопоставление, нахождение первичных рекомендаций)
Модель 2 уровня - RANKER (модель для ранжирования, классификационная модель)

In [99]:
VAL_MATCHER_WEEKS = 5
VAL_RANKER_WEEKS = 3

# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [100]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2136728, 12) Users: 2498 Items: 84180
val_matcher
Shape: (141762, 12) Users: 2097 Items: 25770
train_ranker
Shape: (141762, 12) Users: 2097 Items: 25770
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [101]:
#Проведем префильтрацию данных

n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=10000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 84180 to 10001


In [102]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [103]:
# Добавим параметр категории к исходному обучающему датасету для удобства создания новых фичей
df_join_train_matcher = df_join_train_matcher.merge(item_features[['item_id', 'department']], on='item_id', how='inner')
df_join_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,department
0,2375,26984851516,1,1085983,1,2.99,364,-0.4,1642,1,0.0,0.0,2.99,GROCERY
1,14,27101290145,11,1085983,1,3.39,441,0.0,59,2,0.0,0.0,3.39,GROCERY


#### Оставим только пользователей, которые встречаются в тренировочном датасете, чтобы избежать проблемы холодного старта.

Обработка холодного старта в MainRecommender также добавлена на случай будущих проектов

In [104]:
# ищем общих пользователей
common_users = list(set(data_train_matcher.user_id.values)&(set(data_val_matcher.user_id.values))&set(data_val_ranker.user_id.values))

# оставляем общих пользователей
data_train_matcher = data_train_matcher[data_train_matcher.user_id.isin(common_users)]
data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (787659, 13) Users: 1875 Items: 9996
val_matcher
Shape: (136307, 12) Users: 1875 Items: 25277
train_ranker
Shape: (136307, 12) Users: 1875 Items: 25277
val_ranker
Shape: (115028, 12) Users: 1875 Items: 23962


Построим baseline

In [105]:
# Инициализируем экземпляр класса MainRecommender

recommender = MainRecommender(data_train_matcher, weighting = True)

In [108]:
ACTUAL_COL = 'actual'
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[1005186, 907466, 909497, 940947, 963542, 1067..."
1,6,"[873654, 994928, 1098844, 1122879, 8357613, 98..."


БЕЙЗЛАЙНЫ

In [135]:
N_PREDICT = 30

result_eval_matcher['random_recommendations'] = result_eval_matcher['user_id'].apply(lambda x:\
                                                                    recommender._extend_with_top_popular(recommendations = result_eval_matcher, N = N_PREDICT))

result_eval_matcher['top_popular_recs'] = result_eval_matcher['user_id'].apply(lambda x:\
                                                    recommender._extend_with_top_popular(recommendations = result_eval_matcher, N = N_PREDICT))
result_eval_matcher['weighted_random_recs'] = result_eval_matcher['user_id'].apply(lambda x:\
                                                    recommender._extend_with_top_popular(recommendations = result_eval_matcher, N = N_PREDICT))

In [136]:
#Оценка метрик на безйлайнах
top_k_recall = 30
sorted(calc_recall_at_k(result_eval_matcher, top_k_recall), key=lambda x: x[1],reverse=True)

ValueError: Unable to coerce to Series, length must be 5: given 69

In [137]:
top_k_precision = 5
sorted(calc_precision_at_k(result_eval_matcher, top_k_precision), key=lambda x: x[1],reverse=True)

ValueError: Unable to coerce to Series, length must be 5: given 69

Итак, мы получили значения метрик baseline -ов.
При построении модели будем сравнивать с ними. Чем сильнее результат построенной модели будет превосходить лучший из бейзлайнов (top_popular_items), тем лучше эта модель будет

#### Переходим к построению двухуровневой модели

Построение модели первого уровня Matcher

In [138]:
# Берем в качестве оптимального количества кандидатов 30


N_PREDICT = 30
result_eval_matcher['own_rec'] = make_recommendations(result_eval_matcher, 
                                                       recommender.get_own_recommendations, 
                                                       N_PREDICT=N_PREDICT)

result_eval_matcher['als_rec'] = make_recommendations(result_eval_matcher, 
                                                      recommender.get_als_recommendations, 
                                                      N_PREDICT=N_PREDICT)

result_eval_matcher['bpr_rec'] = make_recommendations(result_eval_matcher, 
                                                      recommender.get_bpr_recommendations, 
                                                      N_PREDICT=N_PREDICT)

result_eval_matcher['bm25_rec'] = make_recommendations(result_eval_matcher, 
                                                      recommender.get_bm25_recommendations, 
                                                      N_PREDICT=N_PREDICT)

result_eval_matcher['tfidf_rec'] = make_recommendations(result_eval_matcher, 
                                                      recommender.get_tfidf_recommendations, 
                                                      N_PREDICT=N_PREDICT)

result_eval_matcher['cosine_rec'] = make_recommendations(result_eval_matcher, 
                                                      recommender.get_cosine_recommendations, 
                                                      N_PREDICT=N_PREDICT)



# #similar_users_recommendation, similar_items_recommendation показывают очень низкий результат
# не будем их учитывать в сравнении

KeyError: 65240.75224736909

In [139]:
result_eval_matcher.head(2)

RecursionError: maximum recursion depth exceeded in __instancecheck__

RecursionError: maximum recursion depth exceeded in __instancecheck__

In [140]:
top_k_recall = 30
sorted(calc_recall_at_k(result_eval_matcher, top_k_recall), key=lambda x: x[1],reverse=True)

ValueError: Unable to coerce to Series, length must be 5: given 69

In [141]:
top_k_precision = 5
sorted(calc_precision_at_k(result_eval_matcher, top_k_precision), key=lambda x: x[1],reverse=True)

ValueError: Unable to coerce to Series, length must be 5: given 69

Очевидно, лучшие результаты показывает модель на основе предыдущих покупок пользователя

##### Подготовка датасета и генерация признаков для модели 2 уровня (модели ранжирования)

In [142]:
# берем пользователей из трейна для модели ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]


# собираем для каждого юзера кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

KeyError: 9754.957653248057

In [143]:
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id' 

KeyError: 'candidates'

In [144]:
# соберем итоговый датафрейм с комбинациями user_id - item_id
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

KeyError: "['candidates'] not found in axis"

In [145]:
df_match_candidates.head(3)

,user_id
0,1827
1,1289
2,1467


In [146]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates


KeyError: 'item_id'

In [147]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # здесь только фактические покупки 

df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

# все, что не фактические покупки - заполняем нулями
df_ranker_train['target'].fillna(0, inplace= True)

df_ranker_train.head()

KeyError: 'item_id'

In [149]:
# Соотношение классов:

df_ranker_train.target.value_counts()

1    136307
Name: target, dtype: int64

In [148]:
# Присоединим к новому тренировочному датасету фичи юзеров и товаров - создадим общий тренировочный датасет

df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,1827,891141,1,3853,PRODUCE,National,VALUE ADDED FRUIT,MELON HALVES/QUARTERS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1827,929388,1,2147,GROCERY,National,ICE CREAM/MILK/SHERBTS,SUPER PREMIUM PINTS,16 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Генерация новых признаков

In [150]:
# Добавим параметр категории к исходному обучающему датасету для удобства создания новых фичей
data_department = data_train_ranker.merge(item_features[['item_id', 'department']], on='item_id', how='inner')
data_department.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,department
0,1827,40702967646,601,891141,2,2.73,33923,0.0,7,87,0.0,0.0,PRODUCE
1,496,40739402373,603,891141,1,1.83,445,0.0,2226,87,0.0,0.0,PRODUCE


In [151]:
# Добавим признаки

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().\
                                        rename('total_item_sales_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().\
                                        rename('total_quantity_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().\
                                        rename('item_freq'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().\
                                        rename('user_freq'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().\
                                        rename('total_user_sales_value'), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().\
                rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().\
                rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().\
            rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().\
            rename('user_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().\
                rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().\
                rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)

Добавление дополнительных признаков

In [152]:
''' ДЛЯ ЮЗЕРОВ '''

# Средний чек
users_sales = data_train_ranker.groupby(USER_COL)['sales_value'].mean().reset_index()
users_sales.rename(columns={'sales_value': 'avg_cheque'}, inplace=True)
df_ranker_train = df_ranker_train.merge(users_sales[['user_id', 'avg_cheque']], on='user_id', how='left')

# Количество уникальных категорий покупателя
users_departments = data_department.groupby(USER_COL)['department'].nunique().reset_index()
users_departments.rename(columns = {'department':'users_unique_departments'}, inplace=True)
df_ranker_train = df_ranker_train.merge(users_departments, on='user_id', how='left')

# Среднее время покупки
bought_time = data_train_ranker.groupby(USER_COL)['trans_time'].mean().reset_index()
bought_time.rename(columns = {'trans_time':'mean_trans_time_by_user'}, inplace=True)
df_ranker_train = df_ranker_train.merge(bought_time, on='user_id', how='left')

# Средний чек корзины 
baskets_sales_value = data_train_ranker.groupby([USER_COL,'basket_id'])['sales_value'].mean().reset_index()
mean_basket_sales_value = baskets_sales_value.groupby(USER_COL)['sales_value'].mean().reset_index()
mean_basket_sales_value.rename(columns = {'sales_value':'mean_sales_value_per_basket'}, inplace=True)
df_ranker_train = df_ranker_train.merge(mean_basket_sales_value, on='user_id', how='left')

# Количество купленных уникальных товаров 
unique_bought_items = data_train_ranker.groupby(USER_COL)[ITEM_COL].nunique().reset_index()
unique_bought_items.rename(columns = {'item_id':'unique_bought_items'}, inplace=True)
df_ranker_train = df_ranker_train.merge(unique_bought_items, on='user_id', how='left')



# Среднее количество уникальных категорий в корзине
users_baskets = data_department.groupby([USER_COL, 'basket_id'])['department'].nunique().reset_index()
users_baskets = users_baskets.groupby(USER_COL)['department'].mean().reset_index()
users_baskets.rename(columns={'department': 'avg_basket_department'}, inplace=True)
df_ranker_train = df_ranker_train.merge(users_baskets[['user_id', 'avg_basket_department']], on='user_id', how='left')

# Средняя сумма покупки в категории
department_sales = data_department.groupby('department')['sales_value'].mean().reset_index()
department_sales.rename(columns={'sales_value': 'mean_sales_value_category'}, inplace=True)
df_ranker_train = df_ranker_train.merge(department_sales, on='department', how='left')

# Средная цена купленных товаров пользователем
users_sales = data_train_ranker.groupby(USER_COL)[['sales_value', 'quantity']].sum().reset_index()
users_sales['avg_price'] = users_sales['sales_value'] / users_sales['quantity']
df_ranker_train = df_ranker_train.merge(users_sales[['user_id', 'avg_price']], on='user_id', how='left')

In [153]:
''' ДЛЯ ТОВАРОВ '''

# Среднее количество покупок товара в неделю
num_purchase_week = data_train_ranker.groupby(ITEM_COL).agg({'week_no': 'nunique', 'quantity': 'sum'}).reset_index()
num_purchase_week['avg_num_purchases_week'] = num_purchase_week['quantity'] / num_purchase_week['week_no']
df_ranker_train = df_ranker_train.merge(num_purchase_week[['item_id', 'avg_num_purchases_week']], on='item_id', how='left')
df_ranker_train['avg_num_purchases_week'].fillna(0, inplace= True)



# Цена товара
items_sales = data_department.groupby(ITEM_COL)[['sales_value', 'quantity']].sum().reset_index()
items_sales['price'] = items_sales['sales_value'] / items_sales['quantity']
items_sales['price'].fillna(0, inplace=True)
df_ranker_train = df_ranker_train.merge(items_sales[['item_id', 'price']], on='item_id', how='left')


# Среднее время покупки товара
bought_item_time = data_train_ranker.groupby(ITEM_COL)['trans_time'].mean().reset_index()
bought_item_time.rename(columns = {'trans_time':'mean_trans_time_by_item'}, inplace=True)
da_ranker_train = df_ranker_train.merge(bought_item_time, on = 'item_id', how = 'left')


# Количество магазинов, где есть товар
items_stores = data_department.groupby(ITEM_COL)['store_id'].sum().reset_index()
items_stores.rename(columns={'store_id': 'n_stores_with_item'}, inplace=True)
items_stores['n_stores_with_item'].fillna(0, inplace = True)
df_ranker_train = df_ranker_train.merge(items_stores, on=ITEM_COL, how='left')

# Количество уникальных магазинов, где есть товар
items_stores = data_department.groupby(ITEM_COL)['store_id'].nunique().reset_index()
items_stores.rename(columns={'store_id': 'n_unique_stores_with_item'}, inplace=True)
items_stores['n_unique_stores_with_item'].fillna(0, inplace = True)
df_ranker_train = df_ranker_train.merge(items_stores, on=ITEM_COL, how='left')

In [154]:
# Построим признак, отражающий средний интервал между покупками пользователя.
users_days = df_join_train_matcher.groupby(USER_COL)['day'].unique().reset_index()
users_days['day'] = users_days['day'].apply(lambda x: sorted(x))
users_days.head()

,user_id,day
0,1,"[51, 67, 88, 94, 101, 108, 111, 128, 137, 146,..."
1,2,"[103, 112, 117, 118, 121, 139, 140, 154, 160, ..."
2,3,"[113, 121, 136, 141, 142, 163, 166, 169, 173, ..."
3,4,"[104, 140, 154, 181, 190, 199, 216, 231, 244, ..."
4,5,"[85, 88, 97, 111, 154, 168, 181, 191, 192, 223..."


In [155]:
def avg_ndays(days):
    diff = 0
    if len(days) > 1:
        for i in range(len(days) - 1):
            diff += days[i+1] - days[i]
        return diff / (len(days) - 1)
    else:
        return 0
    
users_days['avg_interval'] = users_days['day'].apply(avg_ndays)

df_ranker_train = df_ranker_train.merge(users_days[['user_id', 'avg_interval']], on='user_id', how='left')
df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,mean_sales_value_per_basket,unique_bought_items,avg_basket_department,mean_sales_value_category,avg_price,avg_num_purchases_week,price,n_stores_with_item,n_unique_stores_with_item,avg_interval
0,1827,891141,1,3853,PRODUCE,National,VALUE ADDED FRUIT,MELON HALVES/QUARTERS,,NaN,...,2.965362,28,2.4,2.328930,2.208947,3.80,1.428421,74151,16,26.434783
1,1827,929388,1,2147,GROCERY,National,ICE CREAM/MILK/SHERBTS,SUPER PREMIUM PINTS,16 OZ,NaN,...,2.965362,28,2.4,2.545226,2.208947,2.25,3.000000,35436,4,26.434783


Построим признак, в котором будет закодировано место товара в пяти последних покупках клиента.

In [156]:
users_items = data_train_ranker.groupby(USER_COL)[ITEM_COL].apply(list).reset_index()
users_items['item_id'] = users_items['item_id'].apply(lambda x: x[-5:])
users_items.head()

,user_id,item_id
0,1,"[5577022, 8293439, 9526676, 9527558, 10149640]"
1,6,"[1099058, 895268, 1017061, 1082185, 1119051]"
2,7,"[9837501, 12524016, 13072715, 13987153, 13987338]"
3,8,"[924610, 999142, 1080014, 1121694, 1130286]"
4,9,"[7467081, 10150194, 10457112, 12132773, 12171886]"


In [157]:
users_items.loc[users_items['user_id'] == 67, 'item_id'].item()

[1135408, 9194664, 9526666, 10182813, 12385050]

In [158]:
def code_last_sales(x, df=users_items):
    last_sales = df.loc[df['user_id'] == x[0], 'item_id'].item()
    code = str()
    last_sales.reverse()
    for item in last_sales:
        code += '1' if item == x[1] else '0'
    return code

df_ranker_train['Last5sales'] = df_ranker_train[[USER_COL, ITEM_COL]].apply(code_last_sales, axis=1)
df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,unique_bought_items,avg_basket_department,mean_sales_value_category,avg_price,avg_num_purchases_week,price,n_stores_with_item,n_unique_stores_with_item,avg_interval,Last5sales
0,1827,891141,1,3853,PRODUCE,National,VALUE ADDED FRUIT,MELON HALVES/QUARTERS,,NaN,...,28,2.4,2.328930,2.208947,3.80,1.428421,74151,16,26.434783,00000
1,1827,929388,1,2147,GROCERY,National,ICE CREAM/MILK/SHERBTS,SUPER PREMIUM PINTS,16 OZ,NaN,...,28,2.4,2.545226,2.208947,2.25,3.000000,35436,4,26.434783,00000


In [159]:
# Проверим наличие пропусков
df_ranker_train.isnull().sum()

user_id                            0
item_id                            0
target                             0
manufacturer                       0
department                         0
brand                              0
commodity_desc                     0
sub_commodity_desc                 0
curr_size_of_product               0
age_desc                       57522
marital_status_code            57522
income_desc                    57522
homeowner_desc                 57522
hh_comp_desc                   57522
household_size_desc            57522
kid_category_desc              57522
total_item_sales_value             0
total_quantity_value               0
item_freq                          0
user_freq                          0
total_user_sales_value             0
item_quantity_per_week             0
user_quantity_per_week             0
item_quantity_per_basket           0
user_quantity_per_basket           0
item_freq_per_basket               0
user_freq_per_basket               0
a

In [160]:
#Заполним пропуски у количественных признаков
df_ranker_train[['price', 'n_stores_with_item', 'n_unique_stores_with_item']].fillna(0, inplace = True)

/Users/ekaterina/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


Разделение на X_train и y_train и обучение модели

In [161]:
X_train = df_ranker_train.drop(columns = ['target',
                                          'total_quantity_value',
                                          'user_quantity_per_week',
                                          'mean_sales_value_category',
                                          'item_quantity_per_basket',
                                          
                                         ])

#Убрала количественные признаки в весом feature_importances ниже 1 (категориальные оставим)
y_train = df_ranker_train[['target']]

In [162]:
X_train.head(2)

,user_id,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,...,mean_sales_value_per_basket,unique_bought_items,avg_basket_department,avg_price,avg_num_purchases_week,price,n_stores_with_item,n_unique_stores_with_item,avg_interval,Last5sales
0,1827,891141,3853,PRODUCE,National,VALUE ADDED FRUIT,MELON HALVES/QUARTERS,,NaN,NaN,...,2.965362,28,2.4,2.208947,3.80,1.428421,74151,16,26.434783,00000
1,1827,929388,2147,GROCERY,National,ICE CREAM/MILK/SHERBTS,SUPER PREMIUM PINTS,16 OZ,NaN,NaN,...,2.965362,28,2.4,2.208947,2.25,3.000000,35436,4,26.434783,00000


In [163]:
# Обработка категориальных признаков

cat_feats = [
          'department',
         'brand',
         'commodity_desc',
         'sub_commodity_desc',
         'curr_size_of_product',
         'age_desc',
         'marital_status_code',
         'income_desc',
         'homeowner_desc',
         'hh_comp_desc',
         'household_size_desc',
         'kid_category_desc',
         'Last5sales'
    
]


for col in cat_feats:
    X_train[col].fillna(0, inplace=True)

X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'Last5sales']

In [164]:
#Рассчитаем дисбаланс классов - насколько объектов 0 класса больше, чем объектов 1 класса
disbalance = y_train.value_counts()[0] / y_train.value_counts()[1]
disbalance

AttributeError: 'DataFrame' object has no attribute 'value_counts'

Пострение модели

In [165]:
#ПАРАМЕТРЫ БЫЛИ ПОДОБРАНЫ ПОИСКОМ ПО СЕТКЕ, НЕ ВКЛЮЧЕНО В ИТОГОВЫЙ ПРОЕКТ ДЛЯ СОКРАЩЕНИЯ ВРЕМЕНИ РАБОТЫ НОУТБУКА

ctb = CatBoostClassifier(learning_rate=0.1,
                        max_depth=12,
                        n_estimators=550,
                        random_state=42, 
                        cat_features=cat_feats, 
                        class_weights=[1, disbalance],
                        silent=True)

ctb.fit(X_train, y_train)

train_preds = ctb.predict_proba(X_train)

NameError: name 'disbalance' is not defined

In [166]:
# Изучим важность признаков в модели
fi = pd.DataFrame(ctb.feature_importances_, index=X_train.columns, columns=['importance'])
fi.sort_values(by='importance', ascending=False)

NameError: name 'ctb' is not defined

In [167]:
# Оценим качество построенной модели 
df_ranker_predict = df_ranker_train.copy()
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

NameError: name 'train_preds' is not defined

In [168]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]

#Добавляем сначала предсказания (рекомендации) модели 1 уровня
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

KeyError: 65240.75224736909

Сделаем переранжирование рекомендаций на основе результатов классификации модели 2 уровня

In [169]:
def rerank(user_id, N):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].\
            sort_values('proba_item_purchase', ascending=False).head(N).item_id.tolist()

In [170]:
TOPK_PRECISION = 5

result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id:\
                                                            rerank(user_id, N=5))

# Оставляем заведомо больше ранжированных предсказаний для постфильтрации
result_eval_ranker['postfiltered_reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id:\
                                     postfilter_items(rerank(user_id, N=20), item_features = item_features, N=5))
result_eval_ranker['uniquue_reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id:\
                         make_unique_recommendations(rerank(user_id, N=20), N=5))

print(*sorted(calc_precision_at_k(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

KeyError: 'proba_item_purchase'

Удалив дубликаты в df_train_ranker на этапе подготовки датасета, мы сразу получили уникальные товары, поэтому применяя фильтрацию товаров, чтобы остались только уникальные - метрика не меняется, как и товары.

Когда мы делаем постфильтрацию товаров таким образом, чтобы каждый товар был из отдельной категории, метрика существенно падает -> постфильтрация ухудщает результат

#### Проверим, везде ли одинаковое количество рекомендаций
если их кажется меньше, то необходимо будет реализовать дополнение

In [171]:
for num, row in enumerate(result_eval_ranker['own_rec']):
    if len(row) != 30:
        print(num)

KeyError: 'own_rec'

In [172]:
for num, row in enumerate(result_eval_ranker['reranked_own_rec']):
    if len(row) != 5:
        print(num)

KeyError: 'reranked_own_rec'

Оценка на тестовом наборе данных

In [174]:
df_test = pd.read_csv('/Users/ekaterina/Desktop/LEARN/IT/Рекомендательные системы/retail_test1.csv')
df_test.shape

FileNotFoundError: [Errno 2] File /Users/ekaterina/Desktop/LEARN/IT/Рекомендательные системы/retail_test1.csv does not exist: '/Users/ekaterina/Desktop/LEARN/IT/Рекомендательные системы/retail_test1.csv'

In [175]:
df_test = df_test[df_test.user_id.isin(common_users)]

NameError: name 'df_test' is not defined

In [176]:
final_test = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
final_test.columns=[USER_COL, ACTUAL_COL]

NameError: name 'df_test' is not defined

In [177]:
final_test['own_rec'] = final_test[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
final_test['reranked_own_rec'] = final_test[USER_COL].apply(lambda user_id: rerank(user_id, N=5))

print(*sorted(calc_precision_at_k(final_test, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

NameError: name 'final_test' is not defined

In [178]:
# Проверим, есть ли строки где количество предсказаний != 5
for num, row in enumerate(final_test['reranked_own_rec']):
    if len(row) != 5:
        print(num)

NameError: name 'final_test' is not defined

In [179]:
# Сохраним рекомендации
recommendations = final_test[[USER_COL, 'reranked_own_rec']]
recommendations.rename(columns = {'reranked_own_rec' : 'recommendations'}, inplace = True)

NameError: name 'final_test' is not defined

In [180]:
recommendations.to_csv('recommendations.csv', index=False)
recommendations.head(2)

NameError: name 'recommendations' is not defined